# 5. **Torch를 활용한 자연어처리**

In [1]:
!pip install torchtext==0.14.0
!pip install torchdata==0.5.0
!pip install torch==1.13.1
# 코드 실행 이후 Restart_runtime 해주시면 되겠습니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 66.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.17.1
    Uninstalling torchtext-0.17.1:
      Successfully uninstalled torchtext-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tor

In [7]:
import torch
from torch import nn
import torch.optim as optim
from torchtext.datasets import MNLI
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset


In [3]:
# 데이터 생성 및 확인
train_list = list(MNLI(split='train'))
test_list = list(MNLI(split='dev_matched'))

# 0: entailment, 1: neutral, 2: contradiction
tmp = train_list[0]
print(f"label:      {tmp[0]}")
print(f"premise:    {tmp[1]}")
print(f"hypothesis: {tmp[2]}")

# premise, hypothesis를 입력으로 해서 label을 맞추는 작업

label:      1
premise:    Conceptually cream skimming has two basic dimensions - product and geography.
hypothesis: Product and geography are what make cream skimming work. 


In [15]:
def prepare_data():
    train_list = list(MNLI(split='train'))
    test_list = list(MNLI(split='dev_matched'))
    tokenizer = get_tokenizer('basic_english')

    # Build Vocabulary
    def yield_tokens(data_iter):
        for _, premise, hypothesis in data_iter:
            yield tokenizer(premise)
            yield tokenizer(hypothesis)

    vocab = build_vocab_from_iterator(yield_tokens(iter(train_list)), specials=["<unk>", " <sep> "])
    vocab.set_default_index(vocab["<unk>"])

    return train_list, test_list, vocab, tokenizer


In [5]:
# vocab의 역할: 텍스트를 정수입력 형태로 변환
vocab(['here', 'is', 'an', 'example'])

[108, 14, 39, 444]

### **모델 정의하기 (예시)**
모델은 `nn.EmbeddingBag` 레이어와 분류(classification) 목적을 위한 선형 레이어로 구성됩니다.

기본 모드가 “평균(mean)”인 nn.EmbeddingBag 은 임베딩들의 “가방(bag)”의 평균 값을 계산합니다. 이때 텍스트(text) 항목들은 각기 그 길이가 다를 수 있지만, nn.EmbeddingBag 모듈은 *텍스트의 길이를 오프셋(offset)으로 저장하고 있으므로 패딩(padding)이 필요하지는 않습니다.*


![모델](https://tutorials.pytorch.kr/_images/text_sentiment_ngrams_model.png)

In [22]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torch

class MNLIDataset(Dataset):
    def __init__(self, data_list, vocab, tokenizer):
        self.data = data_list
        self.vocab = vocab
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, premise, hypothesis = self.data[idx]
        inputs = premise + ' ' + hypothesis
        inputs = self.vocab(self.tokenizer(inputs))
        return torch.tensor(inputs, dtype=torch.long), torch.tensor(label, dtype=torch.long)

def collate_batch(batch):
    label_list, text_list = [], []
    for _text, _label in batch:
        text_list.append(_text)
        label_list.append(_label)
    text_list = pad_sequence(text_list, batch_first=True, padding_value=0)
    label_list = torch.tensor(label_list, dtype=torch.long)
    return text_list, label_list


In [27]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class=3):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc1 = nn.Linear(embed_dim, embed_dim * 4)
        self.fc2 = nn.Linear(embed_dim * 4, embed_dim * 2)
        self.dropout = nn.Dropout(0.5)  # Dropout layer
        self.fc3 = nn.Linear(embed_dim * 2, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = self.fc1(embedded)
        embedded = self.dropout(embedded)  # Dropout
        embedded = self.fc2(embedded)
        output = self.fc3(embedded)
        return output

### **학습 정의**

In [37]:
train_data, test_data, vocab, tokenizer = prepare_data()
train_dataset = MNLIDataset(train_data, vocab, tokenizer)
test_dataset = MNLIDataset(test_data, vocab, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_batch)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextClassificationModel(len(vocab), 64, num_class=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)


for epoch in range(10):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    scheduler.step()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')


model.eval()
total_acc, total_count = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        total_acc += (outputs.argmax(1) == labels).sum().item()
        total_count += labels.size(0)

print(f"Final Accuracy: {total_acc/total_count * 100:.2f}%")


Epoch 1, Loss: 1.1252681176667496
Epoch 2, Loss: 1.0693405511238974
Epoch 3, Loss: 1.0613488348630757
Epoch 4, Loss: 1.0561046474643605
Epoch 5, Loss: 1.0525861101800569
Epoch 6, Loss: 1.0494393051117463
Epoch 7, Loss: 1.0465786848970078
Epoch 8, Loss: 1.0445157542309906
Epoch 9, Loss: 1.0424967128176343
Epoch 10, Loss: 1.0407595814993749
Final Accuracy: 47.56%


### **학습된 모델 평가**

In [38]:
total_acc, total_count = 0, 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # gpu가속을 위함
model = model.to(device)

for i, data in enumerate(test_list):
    # 데이터 호출
    label = data[0]
    premise = data[1]
    hypothesis = data[2]

    # Forward pass
    inputs = premise + ' ' + hypothesis  # premise와 hypothesis를 모두 고려하기 위한 입력 구성
    inputs = vocab(tokenizer(inputs))  # 텍스트 형태의 입력을 모델이 이해 가능한 형태로 변환 (정수형)
    inputs = torch.as_tensor(inputs, dtype=torch.int32).unsqueeze(0)  # 모델이 받을 수 있는 데이터 형태로 변환
    label = torch.as_tensor(label, dtype=torch.long).unsqueeze(0) # 모델이 받을 수 있는 데이터 형태로 변환

    inputs = inputs.to(device)

    predicted_label = model(inputs)
    predicted_label = predicted_label.detach().cpu()

    total_acc += (predicted_label.argmax(1) == label).sum().item()
    total_count += label.size(0)

    if (i+1) % 1000 == 0:
        print(f'평가 진행중.. [{i+1}/{len(test_list)}]')

print(f"학습된 모델의 최종 정확도: {format(total_acc/total_count * 100, '.3f')} %")

평가 진행중.. [1000/9815]
평가 진행중.. [2000/9815]
평가 진행중.. [3000/9815]
평가 진행중.. [4000/9815]
평가 진행중.. [5000/9815]
평가 진행중.. [6000/9815]
평가 진행중.. [7000/9815]
평가 진행중.. [8000/9815]
평가 진행중.. [9000/9815]
학습된 모델의 최종 정확도: 47.397 %


# **6. 실습 과제**

### - 현재 구현된 자연어처리 모델은 매우 단순한 수준으로 구현돼있습니다.

### - 현재 구현상의 단점들을 극복하고, **모델의 성능을 향상시킬 수 있는 방법들**을 찾아서 적용시키는 것이 과제입니다.

### - 단, 다음 **제한사항**들을 **제외하고**, 그 이외의 방법만을 찾아서 적용해보시기 바랍니다.
- ```test_list```를 학습 데이터로 포함시키는 것
- ```transformers```패키지 등, 사전학습 언어모델을 사용하는 어떠한 종류의 방법

### - 본 실습 자료에 포함된 바, 시도해볼 만한 내용은 다음과 같습니다.
- 모델 Hidden size / layer 개수 변경
- learning rate 변경 / 학습 step 수 증가

### - 본 실습 자료에 포함되지 않았더라도, 성능 개선시킬 수 있는 방법을 찾으신 것이 있다면, 위 제한사항을 제외하고는 모두 적용 가능합니다.

### **42% 이상**의 테스트 정확도가 나오도록 구현하는 것이 과제의 목표입니다.
### 성능이 목표치에 달하지 못하더라도 괜찮으니 최대한 시도해보시기를 바랍니다.

### 구현한 모델, 해당 모델의 학습 코드, 평가 코드가 담긴 **ipynb파일**을 직접 작성하여 제출해주시기 바랍니다.
### 본 실습 자료의 **5.Torch를 활용한 자연어처리** 내 모든 셀을 복사해서 사용하시는걸 권장드립니다.

